In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, time, timedelta
from sklearn.metrics import mean_squared_error
from scipy.integrate import simpson
from numpy import unravel_index
import pvlib
import sys
%run PV_Power_Function.ipynb import PV_Power

In [ ]:
Weather_and_PV_Data = pd.read_excel('C:\\Users\\Lenovo\\OneDrive - Ashesi University\\Desktop\\Master Thesis\\Energy_prod_data/Complete_data.xlsx')
Weather_and_PV_Data.head(5)

In [ ]:
year, month, day = Weather_and_PV_Data['Datetime'].dt.year, Weather_and_PV_Data['Datetime'].dt.month, Weather_and_PV_Data['Datetime'].dt.day
Weather_and_PV_Data[['Year', 'Month', 'Day']] = pd.DataFrame({'Year': year, 
                                                             'Month': month,
                                                             'Day': day})


Weather_and_PV_Data['Date'] = pd.to_datetime(Weather_and_PV_Data[['Year', 'Month', 'Day']])

Weather_and_PV_Data['Time'] = Weather_and_PV_Data['Datetime'].dt.strftime('%H:%M:%S')

In [ ]:
Weather_and_PV_Data.drop(columns=['Year', 'Month', 'Day'], axis = 1, inplace = True)
Weather_and_PV_Data.head(5)

In [ ]:
Weather_and_PV_Data['DayOfYear'] = Weather_and_PV_Data['Datetime'].dt.dayofyear
Weather_and_PV_Data['HourOfDay'] = Weather_and_PV_Data['Datetime'].dt.hour
cols = ['Datetime', 'Date', 'Time', 'DayOfYear', 'HourOfDay', 'GHI ALL SKY', 'CLEAR SKY', 'DNI', 'DHI', 'T2M WET', 'T2M', 'WS10M', 'Internal_Production']
Weather_and_PV_Data = Weather_and_PV_Data[cols]
Weather_and_PV_Data.head(5)

## Modelling Production of the Engineering Section

In [ ]:
Engineering_Building_panels_azimuth = np.array([253.57, 75.22, 294.17, 111.85, 343, 159.21])
Rated_Array_Power_per_azimuth = np.array([25*270, 32*270, 60*270, 81*270, 12*270, 28*270])

Day_oftheYear = np.array(Weather_and_PV_Data['DayOfYear'])
Hour_oftheDay = np.array(Weather_and_PV_Data['HourOfDay'])
GHI_Irradiance = np.array(Weather_and_PV_Data['GHI ALL SKY'])
DNI_ = np.array(Weather_and_PV_Data['DNI'])
DHI_ = np.array(Weather_and_PV_Data['DHI'])
Ambient_Temp = np.array(Weather_and_PV_Data['T2M'])
Wind_Speed = np.array(Weather_and_PV_Data['WS10M'])
Location_lat = 5.7603
Engineering_Building_tilt_angle = 15

Total_Power_Production_KW = np.empty((len(GHI_Irradiance)))
Power_Production_in_azimuth_directions = np.empty((len(GHI_Irradiance), len(Engineering_Building_panels_azimuth)))
Irradiance_in_azimuth_directions = np.empty((len(GHI_Irradiance), len(Engineering_Building_panels_azimuth)))
Temperature_in_azimuth_directions = np.empty((len(GHI_Irradiance), len(Engineering_Building_panels_azimuth)))


Installed_PV_Actual_Production_AC = np.array(Weather_and_PV_Data['Internal_Production'])
Datetimes = np.array(Weather_and_PV_Data['Datetime'])

In [ ]:
Power_OutputArrray_per_azimuth = [0,0,0,0,0,0]
for index1, (day, hour, irradiance, DNI, DHI, temperature, windspeed) in enumerate(zip(Day_oftheYear, Hour_oftheDay, GHI_Irradiance, DNI_, DHI_, Ambient_Temp, Wind_Speed)):
    for index2, (Azimuth, STCpower) in enumerate(zip(Engineering_Building_panels_azimuth, Rated_Array_Power_per_azimuth)):
        Power_OutputArrray_per_azimuth[index2] = PV_Power(hour, day, Location_lat, Engineering_Building_tilt_angle, Azimuth, irradiance, DNI, DHI, STCpower, temperature, windspeed)
        
        Power_Production_in_azimuth_directions[index1, index2] = Power_OutputArrray_per_azimuth[index2][0]
        Irradiance_in_azimuth_directions[index1, index2] = Power_OutputArrray_per_azimuth[index2][1]
        Temperature_in_azimuth_directions[index1, index2] = Power_OutputArrray_per_azimuth[index2][2]
    
     SumOutput1 = 0 
     for i, j, k in Power_OutputArrray_per_azimuth:
         SumOutput1 += np.sum(i)

### Visualization of DC/AC Power Output from the Model and Comparing with the Actual Production @ the Engineering Section

In [ ]:
Model_Production_DC_Dataframe = pd.DataFrame({'Model PV_DC_Power': Total_Power_Production_KW})
Actual_Production_DataFrame = pd.DataFrame({'Datetimes': Datetimes, 'Actual_AC_Power': Installed_PV_Actual_Production_AC})

Pdco = 75/0.98
Model_AC_Power_KW = np.array(pvlib.inverter.pvwatts(Total_Power_Production_KW, Pdco))
Model_Production_AC_Dataframe = pd.DataFrame({'Model PV_AC_Power': Model_AC_Power_KW})

Output_Dataframe = pd.concat([Actual_Production_DataFrame, Model_Production_AC_Dataframe, Model_Production_DC_Dataframe], axis = 1)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(Output_Dataframe['Datetimes'][24:300], Output_Dataframe['Actual_AC_Power'][24:300], color='b', label = 'Actual PV Power')
plt.plot(Output_Dataframe['Datetimes'][24:300], Output_Dataframe['Model PV_AC_Power'][24:300], color='r', label = 'Modelled PV Power')
plt.xlabel('Time (Datetime)')
plt.ylabel('Power (KW)')
plt.xticks(rotation=90)
plt.title('Actual and Modelled PV Power')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(Output_Dataframe['Datetimes'][24:150], Output_Dataframe['Actual_AC_Power'][24:150], color='b', label = 'Actual PV Power')
plt.plot(Output_Dataframe['Datetimes'][24:150], Output_Dataframe['Model PV_AC_Power'][24:150], color='r', label = 'Modelled PV Power')
plt.xlabel('Time (Datetime)')
plt.ylabel('Power (KW)')
plt.xticks(rotation=90)
plt.title('Actual and Modelled PV Power')
plt.legend()
plt.show()

In [ ]:
Model_total_Energy_DC = np.sum(np.array(Output_Dataframe['Model PV_DC_Power']))
#print(f'Total Model DC Energy Output: {round(Model_total_Energy_DC, 2)} KWh')

Model_total_Energy_AC = np.sum(np.array(Output_Dataframe['Model PV_AC_Power']))
print(f'Total Model AC Energy Output: {round(Model_total_Energy_AC, 2)} KWh')

Actual_total_Energy = np.sum(np.array(Output_Dataframe['Actual_AC_Power']))
print(f'Total Actual AC Energy Output: {round(Actual_total_Energy, 2)} KWh')
print('')
print(f'Percent difference between Total Model AC and Actual AC Energy Output: {round(((Model_total_Energy_AC - Actual_total_Energy)/Actual_total_Energy) * 100, 2)}%')
print(f'Mean Squared Error: {mean_squared_error(np.array(Output_Dataframe['Actual_AC_Power']), np.array(Output_Dataframe['Model PV_AC_Power']))}')

## Preprocessing for Visualization

In [ ]:
Month_hours = [720, 744, 144, 576, 720, 744]
monthly_actual_energy_output = []
monthly_modelled_energy_output_AC = []
monthly_modelled_energy_output_DC = []

start = 0
for hours in Month_hours:
    monthly_actual_power_sum = np.sum(np.array(Output_Dataframe['Actual_AC_Power'][start:start + hours]))
    monthly_modelled_AC_power_sum = np.sum(np.array(Output_Dataframe['Model PV_AC_Power'][start:start + hours]))
    monthly_modelled_DC_power_sum = np.sum(np.array(Output_Dataframe['Model PV_DC_Power'][start:start + hours]))
    
    monthly_actual_energy_output.append(monthly_actual_power_sum)
    monthly_modelled_energy_output_AC.append(monthly_modelled_AC_power_sum)
    monthly_modelled_energy_output_DC.append(monthly_modelled_DC_power_sum)
    
    start += hours

## Visualizations

In [ ]:
x = np.arange(6)
Months = ['Nov 2023', 'Dec 2023', 'Jan 2024', 'Mar 2024', 'April 2024', 'May 2024']
bar_width = 0.20
plt.figure(figsize=(12, 7))

plt.bar(x - bar_width, monthly_actual_energy_output, width=bar_width, label='Actual_PV_Production', color='blue')
plt.bar(x, monthly_modelled_energy_output_AC, width=bar_width, label='Modelled_PV_Production_AC', color='green')
plt.bar(x + bar_width, monthly_modelled_energy_output_DC, width=bar_width, label='Modelled_PV_Production_DC', color='red')

plt.xlabel('Months')
plt.ylabel('Energy')
plt.title('Bar Chart of PV Actual vs Modelled Energy Production')
plt.xticks(x, Months)


plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
markers = ['o', 's', '^', '*', 'P', 'X'] 
labels = [
    'Azimuth 1: 253.57°', 'Azimuth 2: 75.22°', 'Azimuth 3: 294.17°', 
    'Azimuth 4: 111.85°', 'Azimuth 5: 343°', 'Azimuth 6: 159.21°']

Num_Panels = np.array([25, 32, 60, 81, 12, 28])

for idx in range(6):
    scatter = plt.scatter(
        Power_Production_in_azimuth_directions[:, idx]/Num_Panels[idx],
        Irradiance_in_azimuth_directions[:, idx],
        c=Temperature_in_azimuth_directions[:, idx],
        cmap='jet',
        s=90,
        alpha=0.75,
        marker=markers[idx],
        label=labels[idx])

cbar = plt.colorbar(scatter)
cbar.set_label('Temperature (°C)')

plt.xlabel('Power Production (W)')
plt.ylabel('Irradiance (W/m²)')
plt.title('Power Production vs. Irradiance Across Azimuth Directions')
plt.legend(title="Azimuth Directions")
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
markers = ['o', 's', '^', '*', 'P', 'X'] 
labels = [
    'Azimuth 1: 253.57° (25 modules)', 'Azimuth 2: 75.22° (32 modules)', 'Azimuth 3: 294.17° (60 modules)', 
    'Azimuth 4: 111.85° (81 modules)', 'Azimuth 5: 343° (12 modules)', 'Azimuth 6: 159.21° (28 modules)']


for idx in range(6):
    scatter = plt.scatter(
        Power_Production_in_azimuth_directions[:, idx],
        Irradiance_in_azimuth_directions[:, idx],
        c=Temperature_in_azimuth_directions[:, idx],
        cmap='jet',
        s=90,
        alpha=0.75,
        marker=markers[idx],
        label=labels[idx])

cbar = plt.colorbar(scatter)
cbar.set_label('Temperature (°C)')

plt.xlabel('Power Production (W)')
plt.ylabel('Irradiance (W/m²)')
plt.title('Power Production vs. Irradiance Across Azimuth Directions')
plt.legend(title="Azimuth Directions")
plt.show()

In [ ]:
print(f'Azimuth_2_max_POA_irradiance: {max(Irradiance_in_azimuth_directions[:,1])}')
print(f'Azimuth_4_max_POA_irradiance: {max(Irradiance_in_azimuth_directions[:,3])}')

In [ ]:
One_year_data = pd.read_excel('C:/Users/Lenovo/OneDrive - Ashesi University/Desktop/Master Thesis/Energy_prod_data/Full_One_Year_Irradiance_Data.xlsx', engine='openpyxl')
One_year_data.head(3)

In [ ]:
Day = np.array(One_year_data['Datetime'].dt.dayofyear)
Hour = np.array(One_year_data['Datetime'].dt.hour)
One_year_GHI_Irradiance = np.array(One_year_data['GHI'])
DNI__ = np.array(One_year_data['DNI'])
DHI__ = np.array(One_year_data['DHI'])
Ambient_Temperature = np.array(One_year_data['Temp'])
WindSpeed = np.array(One_year_data['WS10M'])

In [ ]:
Simulation_Azimuth_angles = np.arange(90, 270, 2)
Simulation_tilt_angles = np.arange(0, 90, 2)
Simulation_STC_Power = 270

In [ ]:
Output_Array = [0]*len(Simulation_Azimuth_angles)
Power_Array = np.empty((len(One_year_GHI_Irradiance), len(Simulation_Azimuth_angles)))
Irradiance_Array = np.empty((len(One_year_GHI_Irradiance), len(Simulation_Azimuth_angles)))
Cell_Temp_Array = np.empty((len(One_year_GHI_Irradiance), len(Simulation_Azimuth_angles)))

MaxPower_in_optimal_tilt_azimuth = np.zeros((len(Simulation_tilt_angles), len(Simulation_Azimuth_angles)))


for i, Simulation_tilt_angle in enumerate(Simulation_tilt_angles):
    for index_1, (day, hour, irradiance, DNI, DHI, temperature, windspeed) in enumerate(zip(Day, Hour, One_year_GHI_Irradiance, DNI__, DHI__, Ambient_Temperature, WindSpeed)):
        for index_2, Azimuth_angle in enumerate(Simulation_Azimuth_angles):
            Output_Array[index_2] = PV_Power(hour, day, Location_lat, Simulation_tilt_angle, Azimuth_angle, irradiance, DNI, DHI, Simulation_STC_Power, temperature, windspeed)
            
            Power_Array[index_1, index_2] = Output_Array[index_2][0]
            Irradiance_Array[index_1, index_2] = Output_Array[index_2][1]
            Cell_Temp_Array[index_1, index_2] = Output_Array[index_2][2]
    
        MaxPower_in_optimal_tilt_azimuth[i, :] = Power_Array.sum(axis = 0)

In [ ]:
optimal_tilt_azimuth_index = unravel_index(MaxPower_in_optimal_tilt_azimuth.argmax(), MaxPower_in_optimal_tilt_azimuth.shape)
print(f'Optimal tilt angle is {Simulation_tilt_angles[optimal_tilt_azimuth_index[0]]}')
print('')
print(f'Optimal azimuth angle is {Simulation_Azimuth_angles[optimal_tilt_azimuth_index[1]]}')

In [ ]:

cmap = plt.colormaps["hsv"]
colors = cmap(np.linspace(0, 1, len(Simulation_Azimuth_angles)))

plt.figure(figsize=(15, 10))

labels = [f"Azimuth {(Simulation_Azimuth_angles[i])}" for i in range(len(Simulation_Azimuth_angles))]
# Loop through each index
for idx in range(len(Simulation_Azimuth_angles)):
    scatter = plt.scatter(
        Power_Array[:, idx],
        Irradiance_Array[:, idx],
        c=[colors[idx]],  
        s=90,
        alpha=0.75,
        label=labels[idx])

plt.xlabel('Power Production (W)')
plt.ylabel('Irradiance (W/m²)')
plt.title('Power Production vs. Irradiance Across Azimuth Directions')

# Legend with all 60 labels
plt.legend(title="Azimuth Directions", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small', ncol=2)
#plt.tight_layout()

# Show plot
plt.show()


In [ ]:
cmap = plt.colormaps["hsv"]
colors = cmap(np.linspace(0, 1, len(Simulation_Azimuth_angles)))

plt.figure(figsize=(15, 10))

labels = [f"Azimuth {(Simulation_Azimuth_angles[i])}" for i in range(len(Simulation_Azimuth_angles))]
# Loop through each index
for idx in range(len(Simulation_Azimuth_angles)):
    scatter = plt.scatter(
        sum(Power_Array[:, idx]/1000),
        sum(Irradiance_Array[:, idx]/1000),
        c=[colors[idx]],  
        s=90,
        alpha=0.75,
        label=labels[idx])

plt.xlabel('Power Production (KW)')
plt.ylabel('Irradiance (KW/m²)')
plt.title('Total Power Production vs. Total Irradiance Across Azimuth Directions')

# Legend with all 60 labels
plt.legend(title="Azimuth Directions", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small', ncol=2)
#plt.tight_layout()

# Show plot
plt.show()

In [ ]:
Total_Power_Array = []
for idx in range(len(Simulation_Azimuth_angles)):
    Total_Power_Array.append(np.sum(Power_Array[:,idx]))

for index, value in enumerate(Total_Power_Array):
    if value == np.max(Total_Power_Array):
        print(f'Max Total Energy Value: {round(value, 2)/1000} KWh ')
        print(f'Angle Corresponding to Max Total Energy Value: {Simulation_Azimuth_angles[index]} degrees')

##### 